In [129]:
%load_ext autoreload
%autoreload 2
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from glob import glob
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
from train_prophet import main_prophet
import datetime
import math
import os
import numpy as np
import pandas as pd

HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/github/data_mining/library")
from parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)

In [290]:
START_DATE = '2017-11-04'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
df_train['datetime'] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_test['datetime'] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_train['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_test['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_train['date'] = df_train['datetime'].map(lambda x: x.date())
df_test['date']  =  df_test['datetime'].map(lambda x: x.date())
df_train['hour'] = df_train['datetime'].map(lambda x: x.hour)
df_test['hour']  =  df_test['datetime'].map(lambda x: x.hour)

In [297]:
def trans_DT(df):
    list_regist = []
    for d, diff in tqdm(df[['date', 'D1']].values):
        if diff < 999999:
            regist = date_add_days(d, -1*diff)
        else:
            regist = date_add_days(d, 0)
        list_regist.append(str(regist))
    df['Regist_date'] = list_regist
    return df
df_train = trans_DT(df_train)
df_test  = trans_DT(df_test)



  0%|          | 0/590540 [00:00<?, ?it/s]

  4%|▍         | 22373/590540 [00:00<00:02, 223728.61it/s]

  8%|▊         | 44960/590540 [00:00<00:02, 224365.70it/s]

 12%|█▏        | 68246/590540 [00:00<00:02, 226846.05it/s]

 15%|█▌        | 90179/590540 [00:00<00:02, 224536.61it/s]

 19%|█▉        | 113833/590540 [00:00<00:02, 228006.23it/s]

 23%|██▎       | 136734/590540 [00:00<00:01, 228306.17it/s]

 27%|██▋       | 159926/590540 [00:00<00:01, 229377.86it/s]

 31%|███       | 182841/590540 [00:00<00:01, 229306.89it/s]

 35%|███▍      | 205925/590540 [00:00<00:01, 229762.29it/s]

 39%|███▉      | 229185/590540 [00:01<00:01, 230605.72it/s]

 43%|████▎     | 252996/590540 [00:01<00:01, 232804.65it/s]

 47%|████▋     | 275843/590540 [00:01<00:01, 231354.36it/s]

 51%|█████     | 298677/590540 [00:01<00:01, 230291.02it/s]

 54%|█████▍    | 321749/590540 [00:01<00:01, 230417.80it/s]

 58%|█████▊    | 345311/590540 [00:01<00:01, 231952.45it/s]

 62%|██████▏   | 368864/590540 [00:01<00:00

In [382]:
train_df['is_train'] = 1
test_df['is_train']  = 0
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [367]:
same_id_2 = [
2999837
,3056355
,3123212
,3133933
,3168396
,3207517
,3229603
,3246696
,3290427
,3325523
,3337729
,3355784
,3430130
,3441776
,3467658
,3511080
,3521314
,3542429
,3551764
,3671539
,3703201
,3733574
,3768559
]
same_id_3 = [
    3668748,
    3693073,
    3698277,
    3709224,
    3664525,
    3729137,
    3757832,
    3705315,
    3705317,
#     3748008,
#     3705320,
#     3705323,
#     3676640,
#     3675467,
#     3700295,
#     3700299,
#     3700300,
#     3700301,
#     3700302,
#     3700303,
#     3667743,
#     3686307,
#     3761431,
#     3716708,
#     3740195,
#     3746885,
#     3670358,
#     3749338,
#     3728126,
#     3737842,
#     3743537,
#     3743586,
#     3744487,
#     3744762,
#     3751017,
#     3751020,
#     3684352,
#     3684367,
#     3729949,
#     3667192,
#     3667227,
#     3667337,
#     3676524,
#     3681076,
#     3705975,
#     3708425,
#     3708754,
#     3708774,
#     3715049,
#     3721365,
#     3730294,
#     3731742,
#     3698518,
#     3730264,
#     3739964,
#     3673044,
#     3698238,
#     3740350,
#     3670750,
#     3700464,
#     3719449,
#     3716834,
#     3716860,
#     3668549,
#     3720667,
#     3714268,
#     3738980,
#     3695883,
#     3733159,
#     3702689,
#     3731050,
#     3702124,
#     3740356,
#     3740380,
#     3754209,
#     3756316,
#     3663768,
#     3686474,
#     3699667,
#     3721623,
#     3701425,
#     3722101,
#     3735782,
#     3757219,
#     3690140,
#     3705555,
#     3730447,
#     3743090,
#     3743094,
#     3746753,
#     3756938,
#     3762422,
#     3691208,
#     3693800,
#     3695850,
#     3699472,
#     3700929,
#     3700953,
#     3703100,
#     3704420,
#     3704437,
#     3712315,
#     3712337,
#     3718525,
#     3736264,
#     3750859,
#     3755848,
#     3755866,
#     3756179,
#     3757001,
#     3758654,
#     3758877,
#     3758881,
#     3759972,
#     3759994,
#     3760000,
#     3694564,
#     3738079,
#     3738141,
#     3738522,
#     3738529,
#     3738539,
#     3741829,
#     3741831,
#     3741833,
#     3747194,
#     3760419,
#     3735759,
#     3701323,
#     3712229,
#     3712789,
#     3739761,
#     3742192,
#     3664053,
#     3681422,
#     3681426,
#     3736224,
#     3751581,
#     3755457,
#     3729864,
#     3738335,  # ここまで Train isFraud=1 ユーザー
#     3675836,  # ここから Train isFraud=1/0 混在ユーザー
#     3669470,
#     3744024,
#     3721865,
#     3741798,
#     3737838,
#     3665398,
#     3744931,
#     3690549,
#     3723397,
#     3734649,
#     3676496,
#     3677414,
#     3682909,
#     3684497,
#     3685731,
#     3694302,
#     3694311,
#     3694324,
#     3696725,
#     3707687,
#     3731212,
#     3732961,
#     3733277,
#     3736311,
#     3751471,
#     3751768,
#     3751770,
#     3698099,
#     3703038,
#     3705750,
#     3733565,
#     3762502,
#     3683366,
#     3712376,
#     3676196,
#     3711864,
#     3726228,
#     3747470,
#     3668357,
#     3693154,
#     3664261,
#     3725072,
#     3763830
]


# data[data[COLUMN_ID].isin(same_id_2)][sorted(list(set(check_cols)-set(['user_id'])))]
# data[data[COLUMN_ID].isin(same_id_3)][sorted(user_keys)]
# data['card5'].value_countse()

異なるuser keyで様々なユーザー特定パターン（正確でなくてもよい）を作り、それぞれのパターンで特徴を作る  
正確でなくてもよい理由としては、粒度を粗くしてもある条件を外した特徴量になるだけなので、問題ない

In [384]:
user_info = [
    'ProductCD',
#     'Regist_date'
]
list_card = [
    'card1',
    'card2',
    'card3',
    'card4',
#     'card5',
#     'card6',
]
list_addr = [
    'addr1',
    'addr2',
]
list_pemail = [
    'P_emaildomain'
]
list_M = [col for col in data.columns if col.startswith('M')]

user_keys__card = user_info + list_card

user_keys__card_addr = user_info + list_card + list_addr
user_keys__card_pemail = user_info + list_card + list_pemail
user_keys__card_M = user_info + list_card + list_M

user_keys__card_addr_pemail = user_info + list_card + list_addr + list_pemail
user_keys__card_addr_M = user_info + list_card + list_addr + list_M
user_keys__card_pemail_M = user_info + list_card + list_pemail + list_M

user_keys__card_addr_pemail_M = user_info + list_card + list_addr + list_pemail + list_M

list_user_keys = [
  ['user_keys__card',      user_keys__card,],
  ['user_keys__card_addr',      user_keys__card_addr,],
  ['user_keys__card_pemail',      user_keys__card_pemail,],
  ['user_keys__card_M',      user_keys__card_M,],
  ['user_keys__card_addr_M',      user_keys__card_addr_M,],
  ['user_keys__card_addr_pemail',      user_keys__card_addr_pemail,],
  ['user_keys__card_pemail_M',      user_keys__card_pemail_M,],
  ['user_keys__card_addr_pemail_M',      user_keys__card_addr_pemail_M,],
]
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', 'datetime']

data['D1'].fillna(-1, inplace=True)
data[user_info + list_card] = data[user_info + list_card].astype('str').fillna('#')

user_keys = user_keys__card_addr
# user_keys = user_keys__card_addr_pemail

uid = 'user_id'
df = data.copy()

for col in user_keys:
    df[col].fillna('#', inplace=True)

df[user_keys] =  df[user_keys].astype('str')

user_keyで粗いグループを作ったら、そのグループの中で同じユーザーを探索し、条件が合えば仲間にしてさらにグループを作っていくイメージ

In [385]:
user_df = df.groupby(user_keys)[COLUMN_ID].min()
user_df = user_df.to_frame(uid)
user_df = df.set_index(user_keys).join(user_df)

check_cols = [COLUMN_ID, uid, COLUMN_TARGET, 'datetime', 'Regist_date', 'is_train'] + ['D1', 'D3', 'C1', 'C13', 'V221', 'V285']
user_df = user_df.reset_index()[check_cols]

if uid not in user_df.columns:
    user_df.reset_index(inplace=True)
    
user_df['hour'] = user_df['datetime'].map(lambda x: x.hour)
user_df['D3'].fillna(-99999, inplace=True)
user_df['C1'].fillna(-99999, inplace=True)
user_df['C13'].fillna(-99999, inplace=True)
user_df['V221'].fillna(-99999, inplace=True)
user_df['V285'].fillna(-99999, inplace=True)

train_max_datetime = train_df['datetime'].max()
test_min_datetime = test_df['datetime'].min()

# user_df.sort_values(by=[uid, 'datetime'], inplace=True)

# user_df['diff_from_regist'] = user_df['datetime'] - user_df['Regist_date'].map(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]) ) )
# user_df['diff_from_regist'] = user_df['diff_from_regist'].map(lambda x: x.days)

# user_df['Regist_date_add_D1'] = user_df[['Regist_date', 'D1']].apply(lambda x: date_add_days(x[0], x[1]), axis=1)
# user_df['Regist_date_add_D1-D3'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]), axis=1)
# user_df['Regist_date_add_D1-D3-1'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]-1), axis=1)

# user_df['date__before1'] = user_df.groupby(uid)['date'].shift(1)
# user_df['date__before2'] = user_df.groupby(uid)['date'].shift(2)
# user_df['date__before3'] = user_df.groupby(uid)['date'].shift(3)
# user_df['D1__before1']   = user_df.groupby(uid)['D1'].shift(1)
# user_df['D1__before2']   = user_df.groupby(uid)['D1'].shift(2)

# user_df['diff_D1__before1'] = user_df[['D1', 'D1__before1']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_D1__before2'] = user_df[['D1', 'D1__before2']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_day__before1'] = user_df[['date', 'date__before1']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before2'] = user_df[['date', 'date__before2']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before3'] = user_df[['date', 'date__before3']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)

# user_df.to_csv('../output/0901_ieee__user_df_card_addr_pemail__for_specific_same_user.csv', index=False)

In [386]:
col_no = 'no'
user_df['no'] = np.arange(user_df.shape[0])+1

tmp = user_df.groupby(uid)['C1'].count().to_frame('user_cnt')
df_multi_user = tmp[tmp['user_cnt']>1]
multi_user = df_multi_user.index.tolist()

list_eval_group = []
list_user_map = []

arg_list = get_parallel_arg_list(n_jobs=16, arg_list=multi_user)

In [387]:
def parallel_explore_same_user(group_no_list):
    
    list_user_map = []
    for group_no in group_no_list:

        group = user_df[user_df[uid]==group_no]
        group.sort_values(by='datetime', inplace=True)
        users = group[col_no].unique().tolist()
        user_map = {}
        
        while True:
        
            remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
        #     remain_user = list( set(users) - set(list(user_map.keys())) )
        
            base_user = remain_user[0]
            user_map[base_user] = base_user
            list_same_user = []
        
            base = group[group[col_no]==base_user]
            base_reg = base['Regist_date'].values[0]
            base_dt = base['datetime'].values[0]
            base_d1 = base['D1'].values[0]
            base_d3 = base['D3'].values[0]
            base_c1 = base['C1'].values[0]
            base_c13 = base['C13'].values[0]
            base_v221 = base['V221'].values[0]
            base_v285 = base['V285'].values[0]
            base_hour = base['hour'].values[0]
            base_fraud = base[COLUMN_TARGET].values[0]
            base_is_train = base['is_train'].values[0]
        
            for i in range(1, len(remain_user), 1):
        
                tmp_user = remain_user[i]
        
                tmp = group[group[col_no]==tmp_user]
                tmp_reg = tmp['Regist_date'].values[0]
                diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)
                
                # Regist_dateで1day以上の差は除外
                if diff_regist_days>1:
                    continue
                
                tmp_dt = tmp['datetime'].values[0]
                tmp_d1 = tmp['D1'].values[0]
                tmp_d3 = tmp['D3'].values[0]
                tmp_c1 = tmp['C1'].values[0]
                tmp_c13 = tmp['C13'].values[0]
                tmp_v221 = tmp['V221'].values[0]
                tmp_v285 = tmp['V285'].values[0]
                tmp_hour = tmp['hour'].values[0]
                tmp_fraud = tmp[COLUMN_TARGET].values[0]
                tmp_is_train = tmp['is_train'].values[0]
                
                is_same_data = base_is_train == tmp_is_train
                
                if not is_same_data:
                    no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
                    if train_max_datetime <= no_term_date <= test_min_datetime:
                        match_d1 = True
                else:
                    match_d1 = True
        
                diff_d1   = tmp_d1 - base_d1
                diff_c1   = abs(tmp_c1 - base_c1)
                minus_c1   = tmp_c1 < base_c1
                if minus_c1:
                    thres_c1 = 3
                    thres_c1_2 = 3
                else:
                    thres_c1 = 4
                    thres_c1_2 = 8
                thres_c1_large = 5
                thres_v = 3
        
                diff_c13  = abs(tmp_c13 - base_c13)
                diff_v221 = abs(tmp_v221 - base_v221)
                diff_v285 = abs(tmp_v285 - base_v285)
        
                ratio_c1   = abs(tmp_c1  / base_c1)
                ratio_c13  = abs(tmp_c13 / base_c13)
        
        #         print("="*12)
        #         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
        #         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
        #         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
        #         print("="*12)
        
                # D3がNullの場合
                if base_d3==-99999 or tmp_d3==-99999:
                    # TODO
                    # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
                    # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい
                    
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    else:
                        continue
                    
        #             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             else:
        #                 continue
        
                # D3がNullsでない場合
                # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
                elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
                    pass
                else:
                    # train testの境目
                    if not is_same_data and match_d1:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                # D3がNullsでない場合
                if base_d3!=-99999 and tmp_d3!=-99999:
                    
                    # 別ユーザーとみなす
                    if diff_c1<-10 or diff_c1>1000:
                        continue
                    if diff_c13<-10 or diff_c13>1000:
                        continue
        
        #             print("  * check diff_V")
                    # V221, 285の変化量が1トランザクションで10以上はないという想定
                    if (10<= diff_v221) and (diff_v221<=100):
                        continue
                    if (10<= diff_v285) and (diff_v285<=100):
                        continue
        
        #             print("  * check diff_C")
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
        #                 print('    * pass1')
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_d1!=0:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                user_map[tmp_user] = base_user
        #         print(pd.Series(user_map))
                
                # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
                base_dt    = tmp['datetime'].values[0]
                base_d1    = tmp['D1'].values[0]
                base_d3    = tmp['D3'].values[0]
                base_c1    = tmp['C1'].values[0]
                base_c13   = tmp['C13'].values[0]
                base_v221  = tmp['V221'].values[0]
                base_v285  = tmp['V285'].values[0]
                base_hour  = tmp['hour'].values[0]
                base_fraud = tmp[COLUMN_TARGET].values[0]
        
            if len(user_map.keys()) == len(users):
                break
                
        tmp_user = pd.Series(user_map).to_frame('same_user')
        tmp_user['group_no'] = same_group
        list_user_map.append(tmp_user)
    return list_user_map

In [389]:
list_p = Parallel(16)([delayed(parallel_explore_same_user)(group_no_list) for group_no_list in arg_list])
list_df = []
for tmp_list in list_p:
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df = pd.concat(list_df, axis=0)

KeyboardInterrupt: 

以下はテスト用

In [368]:
#========================================================================
# テスト用
#========================================================================
tmp_check = user_df[user_df[COLUMN_ID].isin(same_id_3)]
check_user = tmp_check[uid].unique()
check = user_df[user_df[uid].isin(check_user)]
check.shape

(1004, 14)

In [374]:
#========================================================================
# テスト用
#========================================================================

# group = user_df[user_df[uid]==group_no]
group = check
group.sort_values(by='datetime', inplace=True)
users = group[col_no].unique().tolist()
user_map = {}

while True:

    remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
#     remain_user = list( set(users) - set(list(user_map.keys())) )

    base_user = remain_user[0]
    user_map[base_user] = base_user
    list_same_user = []

    base = group[group[col_no]==base_user]
    base_reg = base['Regist_date'].values[0]
    base_dt = base['datetime'].values[0]
    base_d1 = base['D1'].values[0]
    base_d3 = base['D3'].values[0]
    base_c1 = base['C1'].values[0]
    base_c13 = base['C13'].values[0]
    base_v221 = base['V221'].values[0]
    base_v285 = base['V285'].values[0]
    base_hour = base['hour'].values[0]
    base_fraud = base[COLUMN_TARGET].values[0]
    base_is_train = base['is_train'].values[0]

    for i in range(1, len(remain_user), 1):

        tmp_user = remain_user[i]

        tmp = group[group[col_no]==tmp_user]
        tmp_reg = tmp['Regist_date'].values[0]
        diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)
        
        # Regist_dateで1day以上の差は除外
        if diff_regist_days>1:
            continue
        
        tmp_dt = tmp['datetime'].values[0]
        tmp_d1 = tmp['D1'].values[0]
        tmp_d3 = tmp['D3'].values[0]
        tmp_c1 = tmp['C1'].values[0]
        tmp_c13 = tmp['C13'].values[0]
        tmp_v221 = tmp['V221'].values[0]
        tmp_v285 = tmp['V285'].values[0]
        tmp_hour = tmp['hour'].values[0]
        tmp_fraud = tmp[COLUMN_TARGET].values[0]
        tmp_is_train = tmp['is_train'].values[0]
        
        is_same_data = base_is_train == tmp_is_train
        
        if not is_same_data:
            no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
            if train_max_datetime <= no_term_date <= test_min_datetime:
                match_d1 = True
        else:
            match_d1 = True

        diff_d1   = tmp_d1 - base_d1
        diff_c1   = abs(tmp_c1 - base_c1)
        minus_c1   = tmp_c1 < base_c1
        if minus_c1:
            thres_c1 = 3
            thres_c1_2 = 3
        else:
            thres_c1 = 4
            thres_c1_2 = 8
        thres_c1_large = 5
        thres_v = 3

        diff_c13  = abs(tmp_c13 - base_c13)
        diff_v221 = abs(tmp_v221 - base_v221)
        diff_v285 = abs(tmp_v285 - base_v285)

        ratio_c1   = abs(tmp_c1  / base_c1)
        ratio_c13  = abs(tmp_c13 / base_c13)

#         print("="*12)
#         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
#         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
#         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
#         print("="*12)

        # D3がNullの場合
        if base_d3==-99999 or tmp_d3==-99999:
            # TODO
            # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
            # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい
            
            if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
                pass
            else:
                continue
            
#             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
#                 pass
#             else:
#                 continue

        # D3がNullsでない場合
        # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
        elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
            pass
        else:
            # train testの境目
            if not is_same_data and match_d1:
                pass
            else:
                # 別ユーザーとみなす
                continue

        # D3がNullsでない場合
        if base_d3!=-99999 and tmp_d3!=-99999:
            
            # 別ユーザーとみなす
            if diff_c1<-10 or diff_c1>1000:
                continue
            if diff_c13<-10 or diff_c13>1000:
                continue

#             print("  * check diff_V")
            # V221, 285の変化量が1トランザクションで10以上はないという想定
            if (10<= diff_v221) and (diff_v221<=100):
                continue
            if (10<= diff_v285) and (diff_v285<=100):
                continue

#             print("  * check diff_C")
            if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
#                 print('    * pass1')
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_d1!=0:
                pass
            else:
                # 別ユーザーとみなす
                continue

        user_map[tmp_user] = base_user
#         print(pd.Series(user_map))
        
        # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
        base_dt    = tmp['datetime'].values[0]
        base_d1    = tmp['D1'].values[0]
        base_d3    = tmp['D3'].values[0]
        base_c1    = tmp['C1'].values[0]
        base_c13   = tmp['C13'].values[0]
        base_v221  = tmp['V221'].values[0]
        base_v285  = tmp['V285'].values[0]
        base_hour  = tmp['hour'].values[0]
        base_fraud = tmp[COLUMN_TARGET].values[0]

    if len(user_map.keys()) == len(users):
        break

tmp_user = pd.Series(user_map).to_frame('same_user')
check.set_index('no').join(tmp_user['same_user'])

,TransactionID,user_id,isFraud,datetime,Regist_date,is_train,D1,D3,C1,C13,V221,V285,hour,same_user
no,,,,,,,,,,,,,,
4516,2987205,2987205,0.0,2017-11-04 20:56:33,2017-11-04,1,0.0,-99999.0,2.0,2.0,-99999.0,0.0,20,4516
75636,2988069,2988069,1.0,2017-11-05 04:18:16,2017-11-05,1,0.0,-99999.0,40.0,1.0,-99999.0,0.0,4,75636
4517,2989968,2987205,0.0,2017-11-05 13:59:11,2017-11-05,1,0.0,-99999.0,1.0,1.0,-99999.0,0.0,13,4516
75637,2990559,2988069,1.0,2017-11-05 15:31:57,2017-11-05,1,0.0,-99999.0,5.0,0.0,-99999.0,0.0,15,4516
4518,2991265,2987205,0.0,2017-11-05 17:29:19,2017-11-05,1,0.0,-99999.0,1.0,1.0,-99999.0,0.0,17,4518
4519,2992252,2987205,0.0,2017-11-05 20:25:58,2017-11-05,1,0.0,-99999.0,2.0,2.0,-99999.0,0.0,20,4516
4520,2993411,2987205,0.0,2017-11-06 09:59:39,2017-11-06,1,0.0,-99999.0,2.0,2.0,-99999.0,0.0,9,4518
4521,2994121,2987205,1.0,2017-11-06 13:20:42,2017-11-06,1,0.0,-99999.0,34.0,2.0,-99999.0,0.0,13,4521
4522,2994123,2987205,1.0,2017-11-06 13:20:58,2017-11-06,1,0.0,0.0,61.0,5.0,-99999.0,0.0,13,4522


In [376]:
pd.set_option('max_rows', 200)
# テスト用
# check_result = check.set_index('no').join(tmp_user['same_user'])
# check_result.sort_values(by=['same_user', 'datetime'], inplace=True)
# check_result.tail(804).head(200)
check_result[check_result['same_user']==4736]

,TransactionID,user_id,isFraud,datetime,Regist_date,is_train,D1,D3,C1,C13,V221,V285,hour,same_user
no,,,,,,,,,,,,,,
4736,3668748,2987205,NaN,2018-06-06 19:51:27,2018-01-05,0,152.0,153.0,105.0,8.0,3.0,0.0,19,4736
4763,3837824,2987205,NaN,2018-08-13 14:51:38,2018-01-05,0,220.0,68.0,89.0,5.0,0.0,0.0,14,4736
